In [1]:
import os
import glob

from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, IntSlider, FloatSlider

# ====== 경로 설정 ======
CODE_DIR = Path.cwd()
PROJECT_DIR = CODE_DIR.parent              # .../원대한_...
DIP_ROOT = PROJECT_DIR.parent              # .../DIP
DATA_ROOT = DIP_ROOT / "dataset"        # DIP/dataset
RESULTS_VR_DIR = CODE_DIR / "results_vr"

PATHS = {
    "example": os.path.join(DATA_ROOT, "example"),
    "test":    os.path.join(DATA_ROOT, "test"),
}


In [2]:
def load_images(base_path, limit=None, compute_otsu=False):
    """지정 폴더에서 png 이미지 로드 + RGB/V/size (+ Otsu)까지 한 번에 준비"""
    image_paths = sorted(glob.glob(os.path.join(base_path, "*.png")))
    if limit is not None:
        image_paths = image_paths[:limit]

    loaded = []
    for path in image_paths:
        img_bgr = cv2.imread(path)
        if img_bgr is None:
            print(f"[WARN] 읽기 실패: {path}")
            continue

        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)

        item = {
            "path": path,
            "name": os.path.basename(path),
            "rgb": img_rgb,
            "v": v,
            "size": v.size,
        }

        if compute_otsu:
            base_otsu_val, _ = cv2.threshold(
                v, 0, 255,
                cv2.THRESH_BINARY + cv2.THRESH_OTSU
            )
            item["base_otsu"] = int(base_otsu_val)

        loaded.append(item)

    print(f"[INFO] {base_path} 에서 {len(loaded)}개 이미지 로드")
    return loaded


In [3]:
def apply_otsu_and_opening(v_channel, base_otsu, subtract_val, open_kernel_size):
    """V 채널과 Otsu 기준값을 받아서, (Otsu - a) + Inverse + Opening 결과 반환"""
    final_thresh = max(0, base_otsu - subtract_val)

    # 이진화 (Inverse: 어두운 부분을 흰색으로 만들기 위해)
    _, binary_inv = cv2.threshold(
        v_channel, final_thresh, 255,
        cv2.THRESH_BINARY_INV
    )

    # 원형 커널로 Opening (erode -> dilate)
    k = max(1, open_kernel_size)
    kernel_circle = cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (k, k)
    )

    eroded = cv2.erode(binary_inv, kernel_circle, iterations=1)
    opened = cv2.dilate(eroded, kernel_circle, iterations=1)

    return final_thresh, binary_inv, opened


In [4]:
def compute_white_ratio(opened_img, total_size):
    """white pixel 비율(%) 계산"""
    white_pixels = np.count_nonzero(opened_img)
    return (white_pixels / total_size) * 100.0


def detect_large_hole(opened_img, rect_w, rect_h):
    """
    dilation(=OR 연산)을 이용한 '큰 검은 구멍' 탐지.
    rect_w x rect_h 커널을 가로/세로 방향으로 dilation했을 때
    0이 남아 있으면 => 그 크기만큼 완전히 검은 영역이 있다는 뜻.
    """
    rect_w = max(1, rect_w)
    rect_h = max(1, rect_h)

    kernel_rect = cv2.getStructuringElement(
        cv2.MORPH_RECT, (rect_w, rect_h)
    )
    kernel_rect_T = cv2.getStructuringElement(
        cv2.MORPH_RECT, (rect_h, rect_w)
    )

    dilated_1 = cv2.dilate(opened_img, kernel_rect,   iterations=1)
    dilated_2 = cv2.dilate(opened_img, kernel_rect_T, iterations=1)

    has_zero_1 = (np.min(dilated_1) == 0)
    has_zero_2 = (np.min(dilated_2) == 0)

    is_hole_detected = has_zero_1 or has_zero_2

    # 시각화에 쓸 dilation 결과와 suffix도 같이 반환
    show_dilated = dilated_2 if (not has_zero_1 and has_zero_2) else dilated_1
    suffix = "(Rotated)" if (not has_zero_1 and has_zero_2) else "(Normal)"

    return is_hole_detected, show_dilated, suffix


In [5]:
def make_threshold_viewer(loaded_images):
    def update(threshold_value):
        n_imgs = len(loaded_images)
        plt.figure(figsize=(15, 4 * n_imgs))

        for i, item in enumerate(loaded_images):
            _, binary = cv2.threshold(
                item["v"], threshold_value, 255,
                cv2.THRESH_BINARY
            )

            # 1. Original
            plt.subplot(n_imgs, 3, i * 3 + 1)
            plt.imshow(item["rgb"])
            plt.title(f"{item['name']} - Original")
            plt.axis("off")

            # 2. V channel
            plt.subplot(n_imgs, 3, i * 3 + 2)
            plt.imshow(item["v"], cmap="gray")
            plt.title("V channel")
            plt.axis("off")

            # 3. Binary
            plt.subplot(n_imgs, 3, i * 3 + 3)
            plt.imshow(binary, cmap="gray")
            plt.title(f"Threshold = {threshold_value}")
            plt.axis("off")

        plt.tight_layout()
        plt.show()

    interact(
        update,
        threshold_value=IntSlider(
            min=0, max=255, step=1, value=127,
            description="Threshold"
        )
    )


In [6]:
def make_final_inspection_viewer(loaded_images, title_prefix=""):
    def update(subtract_val, open_k, ratio_th, rect_w, rect_h):
        open_k = max(1, open_k)
        rect_w = max(1, rect_w)
        rect_h = max(1, rect_h)

        n_imgs = len(loaded_images)
        plt.figure(figsize=(20, 5 * n_imgs))

        for i, item in enumerate(loaded_images):
            # Step 1: Otsu - α + Opening
            final_thresh, binary_inv, opened = apply_otsu_and_opening(
                item["v"], item["base_otsu"],
                subtract_val, open_k
            )
            ratio = compute_white_ratio(opened, item["size"])
            is_ratio_good = (ratio >= ratio_th)

            # Step 2: 큰 구멍 탐지
            is_hole, dilated, suffix = detect_large_hole(
                opened, rect_w, rect_h
            )

            # 최종 판정
            if not is_ratio_good:
                final_status = "UNGOOD\n(Low Ratio)"
                text_color = "red"
                bg_color = "#ffe6e6"
            elif is_hole:
                final_status = "UNGOOD\n(Big Hole)"
                text_color = "orange"
                bg_color = "#fff5e6"
            else:
                final_status = "GOOD"
                text_color = "green"
                bg_color = "#e6ffe6"

            # --- 시각화 ---

            # 1열: Opening + 비율
            ax1 = plt.subplot(n_imgs, 4, i * 4 + 1)
            plt.imshow(opened, cmap="gray")
            plt.title(
                f"1. Opening & Ratio\n"
                f"White: {ratio:.2f}% (Th: {ratio_th}%)"
            )
            plt.axis("off")

            # 2열: structuring element 모양(가로 rect_w x rect_h)
            ax2 = plt.subplot(n_imgs, 4, i * 4 + 2)
            pad_size = max(rect_w, rect_h) + 2
            vis_kernel = np.zeros((pad_size, pad_size), np.uint8)
            c = pad_size // 2
            cv2.rectangle(
                vis_kernel,
                (c - rect_w//2, c - rect_h//2),
                (c + rect_w//2, c + rect_h//2),
                1, -1
            )
            plt.imshow(vis_kernel, cmap="gray", vmin=0, vmax=1)
            plt.title(f"2. Structuring Element\nRect ({rect_w} x {rect_h})")
            plt.axis("off")

            # 3열: Dilation (OR 결과)
            ax3 = plt.subplot(n_imgs, 4, i * 4 + 3)
            check_msg = "0 Found!" if is_hole else "Full White (1)"
            plt.imshow(dilated, cmap="gray")
            plt.title(f"3. OR Result {suffix}\n{check_msg}")
            plt.axis("off")

            # 4열: 최종 판정 텍스트 박스
            ax4 = plt.subplot(n_imgs, 4, i * 4 + 4)
            ax4.text(
                0.5, 0.6, final_status, fontsize=25, color=text_color,
                ha="center", va="center", fontweight="bold"
            )
            ax4.text(
                0.5, 0.3,
                f"Otsu: {item['base_otsu']} | "
                f"Th: {final_thresh:.0f} (−{subtract_val})",
                fontsize=11, color="black",
                ha="center", va="center"
            )
            ax4.set_xticks([])
            ax4.set_yticks([])
            ax4.set_facecolor(bg_color)

        plt.suptitle(title_prefix, fontsize=14)
        plt.tight_layout()
        plt.show()

    interact(
        update,
        subtract_val=IntSlider(min=0, max=100, step=1, value=0,
                               description="1. Otsu Adj"),
        open_k=IntSlider(min=1, max=21, step=2, value=5,
                         description="2. Open K"),
        ratio_th=FloatSlider(min=0, max=50.0, step=0.5, value=5.0,
                             description="3. Ratio(%)"),
        rect_w=IntSlider(min=10, max=400, step=10, value=150,
                         description="4. Rect W"),
        rect_h=IntSlider(min=10, max=400, step=10, value=50,
                         description="5. Rect H"),
    )


In [7]:
# ===== Example 데이터 =====
example_imgs = load_images(
    PATHS["example"], limit=10, compute_otsu=True
)

print("=== Example: Threshold 실험용 ===")
make_threshold_viewer(example_imgs)

print("=== Example: 최종 판정 파이프라인 확인 ===")
make_final_inspection_viewer(example_imgs, title_prefix="Example Dataset")


# ===== Test 데이터 =====
test_imgs = load_images(
    PATHS["test"], limit=10, compute_otsu=True
)

print("=== Test: 최종 판정 파이프라인 확인 ===")
make_final_inspection_viewer(test_imgs, title_prefix="Test Dataset")


[INFO] c:\Users\CONSOMMES\Downloads\DIP\dataset\example 에서 10개 이미지 로드
=== Example: Threshold 실험용 ===


interactive(children=(IntSlider(value=127, description='Threshold', max=255), Output()), _dom_classes=('widget…

=== Example: 최종 판정 파이프라인 확인 ===


interactive(children=(IntSlider(value=0, description='1. Otsu Adj'), IntSlider(value=5, description='2. Open K…

[INFO] c:\Users\CONSOMMES\Downloads\DIP\dataset\test 에서 10개 이미지 로드
=== Test: 최종 판정 파이프라인 확인 ===


interactive(children=(IntSlider(value=0, description='1. Otsu Adj'), IntSlider(value=5, description='2. Open K…

In [8]:
def classify_image(item, subtract_val, open_k, ratio_th, rect_w, rect_h):
    """
    하나의 이미지(item)에 대해:
    - Otsu - subtract_val 로 threshold 결정
    - inverse binary + opening
    - white 비율 / 큰 구멍 여부 계산
    - 최종 GOOD / UNGOOD 판정
    """
    open_k = max(1, open_k)
    rect_w = max(1, rect_w)
    rect_h = max(1, rect_h)

    # 1) Otsu - a + opening
    final_thresh, binary_inv, opened = apply_otsu_and_opening(
        item["v"], item["base_otsu"],
        subtract_val, open_k
    )

    # 2) white pixel 비율 (퍼센트)
    ratio = compute_white_ratio(opened, item["size"])
    is_ratio_good = (ratio >= ratio_th)

    # 3) 큰 구멍 탐지
    is_hole, dilated, suffix = detect_large_hole(
        opened, rect_w, rect_h
    )

    # 4) 최종 판정
    if not is_ratio_good:
        final_status = "UNGOOD_LOW_RATIO"
    elif is_hole:
        final_status = "UNGOOD_BIG_HOLE"
    else:
        final_status = "GOOD"

    # CSV 등에 쓸 정보들을 dict로 묶어서 반환
    return {
        "name": item["name"],
        "path": item["path"],
        "base_otsu": item["base_otsu"],
        "final_thresh": int(final_thresh),
        "white_ratio": float(ratio),
        "ratio_threshold": float(ratio_th),
        "hole_detected": bool(is_hole),
        "rect_w": int(rect_w),
        "rect_h": int(rect_h),
        "kernel_open": int(open_k),
        "status": final_status,
    }


In [12]:
import os
import csv

# result 폴더 (원하는 위치로 바꿔도 됨)
RESULT_ROOT = CODE_DIR / "results"
os.makedirs(RESULT_ROOT, exist_ok=True)


def export_results_to_csv(
    loaded_images,
    dataset_name,
    subtract_val,
    open_k,
    ratio_th,
    rect_w,
    rect_h,
):
    """
    loaded_images 전체에 대해 classify_image를 적용하고
    result 폴더에 CSV로 저장.
    """
    rows = []

    for item in loaded_images:
        result = classify_image(
            item,
            subtract_val=subtract_val,
            open_k=open_k,
            ratio_th=ratio_th,
            rect_w=rect_w,
            rect_h=rect_h,
        )
        rows.append(result)

    # 저장할 경로: result/example_result.csv 이런 식
    out_path = os.path.join(RESULT_ROOT, f"{dataset_name}_result.csv")

    # CSV 헤더는 dict의 key들로 통일
    fieldnames = [
        "name",
        "path",
        "base_otsu",
        "final_thresh",
        "white_ratio",
        "ratio_threshold",
        "hole_detected",
        "rect_w",
        "rect_h",
        "kernel_open",
        "status",
    ]

    with open(out_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

    print(f"[INFO] CSV 저장 완료: {out_path}")


In [13]:
# 예시: 최종적으로 마음에 드는 파라미터들을 여기 넣기
SUBTRACT_VAL = 10   # Otsu - 10
OPEN_K       = 5    # opening 커널 크기
RATIO_TH     = 5.0  # white 비율 5% 이상이면 pass
RECT_W       = 150  # 큰 구멍 체크용 가로 크기
RECT_H       = 50   # 큰 구멍 체크용 세로 크기

# Example 데이터셋 결과 CSV
export_results_to_csv(
    loaded_images=example_imgs,
    dataset_name="example",
    subtract_val=SUBTRACT_VAL,
    open_k=OPEN_K,
    ratio_th=RATIO_TH,
    rect_w=RECT_W,
    rect_h=RECT_H,
)

# Test 데이터셋 결과 CSV
export_results_to_csv(
    loaded_images=test_imgs,
    dataset_name="test",
    subtract_val=SUBTRACT_VAL,
    open_k=OPEN_K,
    ratio_th=RATIO_TH,
    rect_w=RECT_W,
    rect_h=RECT_H,
)


[INFO] CSV 저장 완료: c:\Users\CONSOMMES\Downloads\DIP\202311322_PSC\Code\results\example_result.csv
[INFO] CSV 저장 완료: c:\Users\CONSOMMES\Downloads\DIP\202311322_PSC\Code\results\test_result.csv
